# Ejercicios de API's
## Enunciado - Obtener datos energéticos usando la API de Red Eléctrica.

Descripción: La empresa para la que trabajamos dispone de una base de datos que contiene información sobre el número de personas censadas en las distintas Comunidades Autónomas españolas desde 1998 hasta 2022. El objetivo es enriquecer estos datos con información sobre la generación de energía renovable y no renovable, utilizando la API de REData, la cual proporciona información sobre el mercado energético en España. Puedes encontrar la documentación de la API aquí: (https://www.ree.es/es/apidatos).

**1. Extracción de Datos a Nivel Nacional:**

Extraer datos de generación de energía renovable y no renovable a nivel nacional desde el 1 de enero de 2011 hasta el 31 de diciembre de 2022. La información debe ser recopilada a nivel mensual.

Transformar los datos para incluir una columna nueva que identifique si la energía es renovable o no renovable.

Almacenar los datos en un DataFrame.

In [6]:
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [60]:
import requests
import pandas as pd

#Formateo la url para crear una función que me permita hacer la misma llamada en diferentes periodos de tiempo. Ya que la API solo ofrece resultados de 24 meses.
url_ReData = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={start_date}&end_date={end_date}&time_trunc=month"
start_date = "2011-01-01T00:00"
end_date = "2012-12-31T23:59"

def obtencion_datos (url_ReData, start_date, end_date):
    respuesta_api_ReData = requests.get(url_ReData)
    if respuesta_api_ReData.status_code != 200:
        print(f"El motivo por el que la llamada falló fue {respuesta_api_ReData.reason}")
    
    return respuesta_api_ReData.json()



In [46]:
#Llamo a la función y obtengo los datos en formato json de los dos primeros años, divividos en meses.
datos_ReData_periodo_2011_2012 = obtencion_datos (url_ReData, "2011-01-01T00:00", "2012-12-31T23:59")
datos_ReData_periodo_2011_2012

{'data': {'type': 'Evolución de la generación renovable/no renovable',
  'id': 'gen2',
  'attributes': {'title': 'Evolución de la generación renovable/no renovable',
   'last-update': '2019-06-12T15:32:24.000+02:00',
   'description': None},
  'meta': {'cache-control': {'cache': 'HIT',
    'expireAt': '2024-05-04T10:04:31'}}},
 'included': [{'type': 'Renovable',
   'id': '10299',
   'groupId': None,
   'attributes': {'title': 'Renovable',
    'description': 'Energía renovable: incluye hidráulica, hidroeólica, eólica, solar fotovoltaica, solar térmica, otras renovables y residuos renovables.',
    'color': '#92d050',
    'type': 'Renovable',
    'magnitude': None,
    'composite': False,
    'last-update': '2019-06-12T15:32:24.000+02:00',
    'values': [{'value': 9918268.536,
      'percentage': 0.3827907635999176,
      'datetime': '2011-01-01T00:00:00.000+01:00'},
     {'value': 7991499.8525,
      'percentage': 0.34539728666246566,
      'datetime': '2011-02-01T00:00:00.000+01:00'},


In [58]:
#Entro en la lista de diccionarios, para poder crear las keys de mi diccionario con los datos limpios.
datos_ReData_periodo_2011_2012.keys()

dict_keys(['data', 'included'])

In [79]:
#Consigo acceder al tipo de energía RENOVAVLE.
datos_ReData_periodo_2011_2012["included"][0]["attributes"]["type"]

'Renovable'

In [133]:
#Consigo acceder al TIPO de energía.
datos_ReData_periodo_2011_2012["included"][0]["attributes"]["description"]

'Energía renovable: incluye hidráulica, hidroeólica, eólica, solar fotovoltaica, solar térmica, otras renovables y residuos renovables.'

In [75]:
#Consigo acceder a un PERIODO de tiempo de un mes.
datos_ReData_periodo_2011_2012["included"][0]["attributes"]["values"][0]["datetime"]

'2011-01-01T00:00:00.000+01:00'

In [77]:
#Consigo acceder al VALOR de un peridodo de mes determinado.
datos_ReData_periodo_2011_2012["included"][0]["attributes"]["values"][0]["value"]

9918268.536

In [84]:
#Consigo acceder al tipo de energía NO RENOVABLE.
datos_ReData_periodo_2011_2012["included"][1]["type"]

'No renovable'

In [92]:
#Consigo acceder a un PERIDODO de tiempo de un mes.
datos_ReData_periodo_2011_2012["included"][1]["attributes"]["values"][0]["datetime"]

'2011-01-01T00:00:00.000+01:00'

In [128]:
#Consigo acceder al VALOR de un peridodo de mes determinado.
datos_ReData_periodo_2011_2012["included"][1]["attributes"]["values"][0]["value"]

15992149.058

In [150]:
#Copio de nuevo la función.
def obtencion_datos (url_ReData, start_date, end_date):
    url_ReData = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={start_date}&end_date={end_date}&time_trunc=month"
    respuesta_api_ReData = requests.get(url_ReData)
    if respuesta_api_ReData.status_code != 200:
        print(f"El motivo por el que la llamada falló fue {respuesta_api_ReData.reason}")  
    return respuesta_api_ReData.json()

#Comienzo a almacenar los datos de forma manual en una lista para tener el global de los datos, pero necesitaba encontrar una forma más sencilla de archivarlos y cree eun bucle for.
datos_ReData_totales = []
start_date = "2011-01-01T00:00"
end_date = "2022-12-31T23:59"

for año in range(2011, 2022):
    start_date = f"{año}-01-01T00:00"
    end_date = f"{año}-12-31T23:59"
    datos_ReData_periodo = obtencion_datos(url_ReData, start_date, end_date)
    if datos_ReData_periodo is not None:
        datos_ReData_totales.append(datos_ReData_periodo)

datos_ReData_totales


[{'data': {'type': 'Evolución de la generación renovable/no renovable',
   'id': 'gen2',
   'attributes': {'title': 'Evolución de la generación renovable/no renovable',
    'last-update': '2019-06-12T15:32:24.000+02:00',
    'description': None},
   'meta': {'cache-control': {'cache': 'HIT',
     'expireAt': '2024-05-04T10:41:37'}}},
  'included': [{'type': 'Renovable',
    'id': '10299',
    'groupId': None,
    'attributes': {'title': 'Renovable',
     'description': 'Energía renovable: incluye hidráulica, hidroeólica, eólica, solar fotovoltaica, solar térmica, otras renovables y residuos renovables.',
     'color': '#92d050',
     'type': 'Renovable',
     'magnitude': None,
     'composite': False,
     'last-update': '2019-06-12T15:32:24.000+02:00',
     'values': [{'value': 9918268.536,
       'percentage': 0.3827907635999176,
       'datetime': '2011-01-01T00:00:00.000+01:00'},
      {'value': 7991499.8525,
       'percentage': 0.34539728666246566,
       'datetime': '2011-02-01

In [153]:
def diccionario_datos_ReData(datos_ReData_totales, diccionario_energia):
    
    for data in datos_ReData_totales:
        for datos in data["included"]:
            tipo_energia = datos["attributes"]["description"]
            fecha = datos["attributes"]["values"][0]["datetime"]
            valor = datos["attributes"]["values"][0]["value"]
            
            # Determinar en qué diccionario almacenar los datos según el tipo de energía
            if datos["type"] == "Renovable":
                diccionario_energia["tipo_energia"].append(tipo_energia)
                diccionario_energia["fecha"].append(fecha)
                diccionario_energia["valor"].append(valor)
                diccionario_energia["tipo_generacion"].append("Renovable")
            elif datos["type"] == "No renovable":
                diccionario_energia["tipo_energia"].append(tipo_energia)
                diccionario_energia["fecha"].append(fecha)
                diccionario_energia["valor"].append(valor)
                diccionario_energia["tipo_generacion"].append("No renovable")

    return diccionario_energia

energia_total = {"tipo_energia": [], "fecha": [], "valor": [], "tipo_generacion": []}
diccionario_energia = diccionario_datos_ReData(datos_ReData_totales, energia_total)

df_diccionario_energia_total = pd.DataFrame(diccionario_energia)
df_diccionario_energia_total

,tipo_energia,fecha,valor,tipo_generacion
0,"Energía renovable: incluye hidráulica, hidroeó...",2011-01-01T00:00:00.000+01:00,9.918269e+06,Renovable
1,None,2011-01-01T00:00:00.000+01:00,1.599215e+07,No renovable
2,"Energía renovable: incluye hidráulica, hidroeó...",2012-01-01T00:00:00.000+01:00,6.312121e+06,Renovable
3,None,2012-01-01T00:00:00.000+01:00,1.897961e+07,No renovable
4,"Energía renovable: incluye hidráulica, hidroeó...",2013-01-01T00:00:00.000+01:00,1.030887e+07,Renovable
5,None,2013-01-01T00:00:00.000+01:00,1.448878e+07,No renovable
6,"Energía renovable: incluye hidráulica, hidroeó...",2014-01-01T00:00:00.000+01:00,1.171991e+07,Renovable
7,None,2014-01-01T00:00:00.000+01:00,1.231336e+07,No renovable
8,"Energía renovable: incluye hidráulica, hidroeó...",2015-01-01T00:00:00.000+01:00,8.571094e+06,Renovable
9,None,2015-01-01T00:00:00.000+01:00,1.602089e+07,No renovable


**2. Extracción de Datos por Comunidad Autónoma:**

Extraer datos de generación de energía renovable y no renovable por Comunidad Autónoma.

Utilizar el diccionario cod_comunidades para obtener los códigos de cada comunidad autónoma.

Transformar los datos para incluir una columna que especifique la comunidad autónoma y su identificador.

Almacenar los datos en un DataFrame.